## Setup configuration file and parameters
Prepare or chose a config file and set the parameters to your needs

In [2]:
## Since spicelib>1.1.2 leads to problematic behaviour please confirm the version
from importlib.metadata import version
print(version('PyLTSpice'))
print(version('spicelib'))

5.3.1
1.1.2


In [6]:
import json
from src.FunctionalValidation import FunctionalValidation
from src.ControlLogicGenerator import ControlLogicGenerator
from src.Simulator import Simulator
from src.Plotter import Plotter
from src.util import copy_pwm_files

############## PARAMETERS ###################
max_dev = 40    # Default 40
dev_wf = 20     # Default 20
fig_type = ['pdf', 'png', 'svg'][1]
config_file = './configs/PPU1.json'

with open(config_file, 'r') as f:
    config = json.load(f)

Verify the functionality of the algorithm and present a state history of each memristor:

In [7]:
# Check if the algorithm is valid and the resulting states are correct
Verifier = FunctionalValidation(config) 
Verifier.calc_algorithm(plot_tt=True)
print("\n--------------- Logic States verified! ---------------\n")


--------------- Logic States verified! ---------------



## Automatically create PWM signals
Instantiate a PWMWriter class, automatically write PWM files for the chosen algorithm in the defined topology, and temporary store them in the **PWM_output** 
folder for debugging:


In [4]:
# Automatically create PWM signals and store them in "PWM_output"
CLG = ControlLogicGenerator(config)
CLG.eval_algo()
print("\n--------------- PWM Signals created! ---------------\n")

File ./outputs/PWM_output/a.csv created!
File ./outputs/PWM_output/b.csv created!
File ./outputs/PWM_output/c.csv created!
File ./outputs/PWM_output/w1.csv created!
File ./outputs/PWM_output/w2.csv created!
File ./outputs/PWM_output/a_sw.csv created!
File ./outputs/PWM_output/b_sw.csv created!
File ./outputs/PWM_output/c_sw1.csv created!
File ./outputs/PWM_output/c_sw2.csv created!
File ./outputs/PWM_output/w1_sw1.csv created!
File ./outputs/PWM_output/w1_sw2.csv created!
File ./outputs/PWM_output/w2_sw1.csv created!
File ./outputs/PWM_output/w2_sw2.csv created!
PWM Files written successfully

--------------- PWM Signals created! ---------------



Overwrite the current algorithm in the chosen topology folder:

In [5]:
# Copy the files to the folder of the corresponding topology (This removes the old files !)
copy_pwm_files(config, CLG.step_size)
print(f"""\n--------- Files of {config["topology"]} topology overwritten! --------\n""")


--------- Files of Semi-Serial topology overwritten! --------



## Illustrate the deviation experiments
Instantiate a Plotter class in which the Simulator is embedded for convenience. \
The energy consumption is automatically calculated.

In [6]:
PLT = Plotter(config)

Calculating energy consumption:


100%|██████████| 8/8 [00:08<00:00,  1.02s/it]

Average Energy consumption: 1.7239985999999998e-09
Energy over Combination: [1.1116853999999999e-09, 1.5245297999999998e-09, 1.4622498000000001e-09, 1.863126e-09, 1.5319817999999997e-09, 2.0664719999999996e-09, 1.871658e-09, 2.360286e-09]


The three main plot options are:
1) plot_deviation_scatter()
2) plot_deviation_range()
3) plot_waveforms_with_deviation()

If the simulations have been run already (The files of the last simulation is stored in the folder **outputs**), the parameter **recompute** can be set **False**.

In [ ]:
PLT.plot_deviation_scatter(max_dev=max_dev, recompute=True, fig_type=fig_type)
PLT.plot_deviation_range(max_dev=max_dev, recompute=False, fig_type=fig_type, save_dev_range=True)
print(f"\n--------- Deviation Experiments completed --------\n")

Create waveform images with deviation ranges for all input combinations with deviation **dev_wf**:

In [ ]:
for comb in range(2**len(config["inputs"])):
    comb_str = bin(comb)[2:].zfill(len(config["inputs"]))
    PLT.plot_waveforms_with_deviation(comb_str, dev=dev_wf, recompute=False, fig_type=fig_type)
print(f"\n--------- Waveforms with deviation {dev_wf} saved --------\n")

Illustrate a single input combination with unique parameters:

In [ ]:
comb = '1010'
dev = 20
fig_type_s = 'png'
recompute_s = True
PLT.plot_waveforms_with_deviation(comb, dev=dev, recompute=recompute_s, fig_type=fig_type_s, plots_per_subfigure=4)

## Store the results
Save created images and state deviation files in **OUTPUT_FILES**


In [ ]:
PLT.save_algorithm_files(f"""{config["algorithm"].split(".")[0]}""")